<a href="https://colab.research.google.com/github/surasakcho/A_A/blob/master/01_07_colab_ls8_split_pixel_values_raw_prov_cd_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries & Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install rasterio
!pip install geopandas


     |████████████████████████████████| 18.1MB 222kB/s 
     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 10.4MB 10.9MB/s 
     |████████████████████████████████| 14.7MB 52.4MB/s 


In [0]:
import numpy as np
import pandas as pd
import os, shutil
from glob import glob
import re
import datetime
from datetime import datetime
import random
from tqdm import tqdm_notebook
import pickle
import matplotlib.pyplot as plt
import geopandas as gpd
from pyproj import CRS
import rasterio
from rasterio import plot
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling
from rasterio.mask import mask
from shapely.geometry import Polygon, mapping
from datetime import datetime, timedelta
%matplotlib inline

# Define functions

In [0]:

def list_files_re(rootpath, filename_re=None, folder_re=None ):
    '''
    rootpath : root path to lookup files
    filename_re : regular expression to search for filename
    folder_re : regular expression to search for folder

    return : a list of filepaths
    '''


    list_files = []
    for folder, _, files in os.walk(rootpath):
        for file in files:     
            if filename_re == None:
                filename_re = '.*'
            if folder_re == None:
                folder_re = '.*'
                
            if ((re.search(filename_re, file) != None) & (re.search(folder_re, folder) != None)):
                list_files.append(os.path.join(folder, file))
        
    return list_files    
        
        
    

In [0]:

def read_pickle(in_pickle_path):
    '''
    Read a pickle file.

    Parameters
    ----------------------------------
    in_pickle_path : path of pickle to be read

    Return
    ----------------------------------
    An object / variable of read pickle file
    '''

    pkl_file = open(in_pickle_path, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()

    return data

# Define paths & parameters

In [0]:
CYCLE_DAYS = 16


my_drive_path = "/content/drive/My Drive/"

landsat_folder = os.path.join(my_drive_path, '!landsat8-prep')

pixel_rowcol_map_province_folder = os.path.join(landsat_folder, 'pixel_rowcol_map_province')
pixel_rowcol_map_tambon_folder = os.path.join(landsat_folder, 'pixel_rowcol_map_tambon')

pixel_values_raw_folder = os.path.join(landsat_folder, 'pixel_values_raw')
pixel_values_raw_prov_folder = os.path.join(landsat_folder, 'pixel_values_raw_prov')
os.makedirs(pixel_values_raw_prov_folder, exist_ok=True)

# Load data

In [0]:
df_province_cd = pd.read_csv(os.path.join(landsat_folder, 'DOAE_PROVINCE.csv'))

df_province_cd = df_province_cd.sample(frac=1.0)


# Split pixel values by prov_cd

In [0]:
for prov_cd in tqdm_notebook(df_province_cd['PROVINCE_CODE']):    
    #prov_cd = 19
    print(f'Processing : p{prov_cd}')
    
    #get list of province row col map of prov_cd
    list_pixel_rowcol_map_province_path = list_files_re(pixel_rowcol_map_province_folder, f'df_pixel_map_p{prov_cd}_\d*.parquet')
    #shuffle for multi-session processing
    random.shuffle(list_pixel_rowcol_map_province_path)
    
    for pixel_rowcol_map_province_path in list_pixel_rowcol_map_province_path:
        #iterate each scene in prov_cd
        scene_id = pixel_rowcol_map_province_path.split('.')[0][-6:]


        out_path = os.path.join(pixel_values_raw_prov_folder, f'df_pixel_values_raw_p{prov_cd}_{scene_id}.parquet')  
        #check whether file already exists
        if os.path.exists(out_path):
            continue

        print(f'Loading scene_id : {scene_id}')
        df_prov_scene_rowcol_map = pd.read_parquet(pixel_rowcol_map_province_path).set_index(['scene_id', 'row', 'col'])
        df_tambon_scene_rowcol_map = pd.read_parquet(os.path.join(pixel_rowcol_map_tambon_folder, f'df_pixel_map_tambon_{scene_id}.parquet')).set_index(['scene_id', 'row', 'col'])

        df_pixel_values_prov_scene = df_prov_scene_rowcol_map.merge(df_tambon_scene_rowcol_map, how='inner', left_index=True, right_index=True)

        #Merge all years into one dataframe
        list_pixel_values_raw_path = list_files_re(pixel_values_raw_folder, f'df_pixel_values_{scene_id}_(2015|2016|2017|2018|2019).parquet')
        
        continue_next_scene = False
        
        for pixel_values_raw_path in list_pixel_values_raw_path:            
            #iterate each year
            year = pixel_values_raw_path.split('.')[0][-4:]

            #If current scene is created by other session, skip it.
            if os.path.exists(out_path):
                continue_next_scene = True
                break
            else:    
                print(f'Loading year : {year}')    

            #join another year to existing df
            df_tmp = pd.read_parquet(pixel_values_raw_path).set_index(['scene_id', 'row', 'col'])
            df_pixel_values_prov_scene = df_pixel_values_prov_scene.merge(df_tmp, how='inner', left_index=True, right_index=True)
            del(df_tmp)
        
        
        if continue_next_scene:
            continue
        
        df_pixel_values_prov_scene = df_pixel_values_prov_scene.reset_index().set_index(['scene_id', 'row', 'col', 'prov_cd', 'tambon_key'], drop=True)        
        df_pixel_values_prov_scene.columns = [col[-8:] for col in df_pixel_values_prov_scene.columns]

        start_date = datetime.strptime(min(df_pixel_values_prov_scene.columns), '%Y%m%d')
        last_date = datetime.strptime(max(df_pixel_values_prov_scene.columns), '%Y%m%d')

        list_complete_dates = []

        for i in range(int((last_date - start_date).days / CYCLE_DAYS)):
            list_complete_dates.append(datetime.strftime(start_date + timedelta(i * CYCLE_DAYS), '%Y%m%d'))
        
        df_pixel_values_prov_scene= df_pixel_values_prov_scene.reindex(columns=list_complete_dates)

        nbr_columns = len(df_pixel_values_prov_scene.columns)

        df_pixel_values_prov_scene['zeros'] = np.where(df_pixel_values_prov_scene.values == 0, 1, 0).sum(axis=1)
        df_pixel_values_prov_scene['nans'] = np.where(np.isnan(df_pixel_values_prov_scene.values), 1, 0).sum(axis=1)
        df_pixel_values_prov_scene['pct_missing'] = (df_pixel_values_prov_scene['zeros'].values + df_pixel_values_prov_scene['nans'].values) / nbr_columns
        #remove pixel with > 20% zeros or missing
        df_pixel_values_prov_scene = df_pixel_values_prov_scene[df_pixel_values_prov_scene['pct_missing'] <= 0.2]

        #recheck whether there is the same file created between processing
        if ~os.path.exists(out_path):
            df_pixel_values_prov_scene.to_parquet(out_path)

        print(f'scene_id : {scene_id} done')
        print()
    print('############')        
    print()

Processing : p80
Loading scene_id : 130053
Loading year : 2015
Loading year : 2016
Loading year : 2017
Loading year : 2018
Loading year : 2019
scene_id : 130053 done

Loading scene_id : 130054
Loading year : 2015
Loading year : 2016
Loading year : 2017
Loading year : 2018
Loading year : 2019
scene_id : 130054 done

Loading scene_id : 128054
Loading year : 2015
Loading year : 2016
Loading year : 2017
Loading year : 2018
Loading year : 2019
scene_id : 128054 done

Loading scene_id : 129055
Loading year : 2015
Loading year : 2016
Loading year : 2017
Loading year : 2018
Loading year : 2019
scene_id : 129055 done

Loading scene_id : 128055
Loading year : 2015
Loading year : 2016
Loading year : 2017
Loading year : 2018
Loading year : 2019
scene_id : 128055 done

Loading scene_id : 129053
Loading year : 2015
Loading year : 2016
Loading year : 2017
Loading year : 2018
Loading year : 2019
scene_id : 129053 done

############

Processing : p64
Loading scene_id : 130047


# Test

In [28]:
df_pixel_values_prov_scene = df_pixel_values_prov_scene.reset_index().set_index(['scene_id', 'row', 'col', 'prov_cd', 'tambon_key'], drop=True)        
df_pixel_values_prov_scene.columns = [col[-8:] for col in df_pixel_values_prov_scene.columns]

start_date = datetime.strptime(min(df_pixel_values_prov_scene.columns), '%Y%m%d')
last_date = datetime.strptime(max(df_pixel_values_prov_scene.columns), '%Y%m%d')

list_complete_dates = []

for i in range(int((last_date - start_date).days / CYCLE_DAYS)):
    list_complete_dates.append(datetime.strftime(start_date + timedelta(i * CYCLE_DAYS), '%Y%m%d'))

df_pixel_values_prov_scene= df_pixel_values_prov_scene.reindex(columns=list_complete_dates)

nbr_columns = len(df_pixel_values_prov_scene.columns)

df_pixel_values_prov_scene['zeros'] = np.where(df_pixel_values_prov_scene.values == 0, 1, 0).sum(axis=1)
df_pixel_values_prov_scene['nans'] = np.where(np.isnan(df_pixel_values_prov_scene.values), 1, 0).sum(axis=1)
df_pixel_values_prov_scene['pct_missing'] = (df_pixel_values_prov_scene['zeros'].values + df_pixel_values_prov_scene['nans'].values) / nbr_columns
#remove pixel with > 20% zeros or missing
df_pixel_values_prov_scene = df_pixel_values_prov_scene[df_pixel_values_prov_scene['pct_missing'] <= 0.2]

#recheck whether there is the same file created between processing
if ~os.path.exists(out_path):
    df_pixel_values_prov_scene.to_parquet(out_path)

print(f'scene_id : {scene_id} done')
print()

scene_id : 129054 done



In [20]:
df_tmp = df_tambon_scene_rowcol_map.copy()

df_tmp.reset_index().set_index(['scene_id', 'row', 'col', 'tambon_key'], drop=True)

Empty DataFrame
Columns: []
Index: [(128054, 5940, 195, 3194), (128054, 5940, 196, 3194), (128054, 5940, 197, 3194), (128054, 5940, 198, 3194), (128054, 5941, 191, 3194), (128054, 5941, 192, 3194), (128054, 5941, 193, 3194), (128054, 5941, 194, 3194), (128054, 5941, 195, 3194), (128054, 5941, 196, 3194), (128054, 5941, 197, 3194), (128054, 5941, 198, 3194), (128054, 5941, 199, 3194), (128054, 5941, 200, 3194), (128054, 5942, 189, 3194), (128054, 5942, 190, 3194), (128054, 5942, 191, 3194), (128054, 5942, 192, 3194), (128054, 5942, 193, 3194), (128054, 5942, 194, 3194), (128054, 5942, 195, 3194), (128054, 5942, 196, 3194), (128054, 5942, 197, 3194), (128054, 5942, 198, 3194), (128054, 5942, 199, 3194), (128054, 5942, 200, 3194), (128054, 5942, 201, 3194), (128054, 5943, 188, 3194), (128054, 5943, 189, 3194), (128054, 5943, 190, 3194), (128054, 5943, 191, 3194), (128054, 5943, 192, 3194), (128054, 5943, 193, 3194), (128054, 5943, 194, 3194), (128054, 5943, 195, 3194), (128054, 5943, 196, 3194), (128054, 5943, 197, 3194), (128054, 5943, 198, 3194), (128054, 5943, 199, 3194), (128054, 5943, 200, 3194), (128054, 5943, 201, 3194), (128054, 5943, 202, 3194), (128054, 5944, 186, 3194), (128054, 5944, 187, 3194), (128054, 5944, 188, 3194), (128054, 5944, 189, 3194), (128054, 5944, 190, 3194), (128054, 5944, 191, 3194), (128054, 5944, 192, 3194), (128054, 5944, 193, 3194), (128054, 5944, 194, 3194), (128054, 5944, 195, 3194), (128054, 5944, 196, 3194), (128054, 5944, 197, 3194), (128054, 5944, 198, 3194), (128054, 5944, 199, 3194), (128054, 5944, 200, 3194), (128054, 5944, 201, 3194), (128054, 5944, 202, 3194), (128054, 5945, 186, 3194), (128054, 5945, 187, 3194), (128054, 5945, 188, 3194), (128054, 5945, 189, 3194), (128054, 5945, 190, 3194), (128054, 5945, 191, 3194), (128054, 5945, 192, 3194), (128054, 5945, 193, 3194), (128054, 5945, 194, 3194), (128054, 5945, 195, 3194), (128054, 5945, 196, 3194), (128054, 5945, 197, 3194), (128054, 5945, 198, 3194), (128054, 5945, 199, 3194), (128054, 5945, 200, 3194), (128054, 5945, 201, 3194), (128054, 5945, 202, 3194), (128054, 5945, 203, 3194), (128054, 5946, 186, 3194), (128054, 5946, 187, 3194), (128054, 5946, 188, 3194), (128054, 5946, 189, 3194), (128054, 5946, 190, 3194), (128054, 5946, 191, 3194), (128054, 5946, 192, 3194), (128054, 5946, 193, 3194), (128054, 5946, 194, 3194), (128054, 5946, 195, 3194), (128054, 5946, 196, 3194), (128054, 5946, 197, 3194), (128054, 5946, 198, 3194), (128054, 5946, 199, 3194), (128054, 5946, 200, 3194), (128054, 5946, 201, 3194), (128054, 5946, 202, 3194), (128054, 5946, 203, 3194), (128054, 5947, 150, 3194), (128054, 5947, 151, 3194), (128054, 5947, 186, 3194), (128054, 5947, 187, 3194), (128054, 5947, 188, 3194), ...]

[3964869 rows x 0 columns]

In [16]:
df_prov_scene_rowcol_map

prov_cd
scene_id row  col         
128054   4174 2         80
              3         80
              4         80
         4175 2         80
              3         80
...                    ...
         6958 224       80
              225       80
              226       80
              227       80
              228       80

[2400298 rows x 1 columns]

prov_cd  tambon_key
scene_id row  col                     
128054   4174 2         80        3266
              3         80        3266
              4         80        3266
         4175 2         80        3266
              3         80        3266
...                    ...         ...
         6958 226       80        3943
              227       80        3197
              227       80        3943
              228       80        3197
              228       80        3943

[2433205 rows x 2 columns]

In [13]:
df_tambon_scene_rowcol_map

tambon_key
scene_id row  col            
128054   5940 195        3194
              196        3194
              197        3194
              198        3194
         5941 191        3194
...                       ...
         1120 55         6357
              57         6357
              60         6357
              61         6357
              62         6357

[3964869 rows x 1 columns]

20150113  20150129  20150214  ...  20191108  20191124  20191210
scene_id row col                                ...                              
128051   920 10        0.0       0.0       0.0  ...       0.0       0.0       0.0
         928 14        0.0       0.0       0.0  ...       0.0       0.0       0.0
         911 3         0.0       0.0       0.0  ...       0.0       0.0       0.0
         936 34        0.0       0.0       0.0  ...       0.0       0.0       0.0
         917 11        0.0       0.0       0.0  ...       0.0       0.0       0.0
...                    ...       ...       ...  ...       ...       ...       ...
         907 21        0.0       0.0       0.0  ...       0.0       0.0       0.0
         925 8         0.0       0.0       0.0  ...       0.0       0.0       0.0
         921 21        0.0       0.0       0.0  ...       0.0       0.0       0.0
         927 17        0.0       0.0       0.0  ...       0.0       0.0       0.0
         915 14        0.0       0.0       0.0  ...       0.0       0.0       0.0

[655 rows x 113 columns]

In [27]:
list_date

[datetime.datetime(2015, 1, 13, 0, 0),
 datetime.datetime(2015, 1, 29, 0, 0),
 datetime.datetime(2015, 2, 14, 0, 0),
 datetime.datetime(2015, 3, 2, 0, 0),
 datetime.datetime(2015, 3, 18, 0, 0),
 datetime.datetime(2015, 4, 3, 0, 0),
 datetime.datetime(2015, 4, 19, 0, 0),
 datetime.datetime(2015, 5, 5, 0, 0),
 datetime.datetime(2015, 5, 21, 0, 0),
 datetime.datetime(2015, 6, 6, 0, 0),
 datetime.datetime(2015, 6, 22, 0, 0),
 datetime.datetime(2015, 7, 8, 0, 0),
 datetime.datetime(2015, 7, 24, 0, 0),
 datetime.datetime(2015, 8, 9, 0, 0),
 datetime.datetime(2015, 8, 25, 0, 0),
 datetime.datetime(2015, 9, 10, 0, 0),
 datetime.datetime(2015, 9, 26, 0, 0),
 datetime.datetime(2015, 10, 12, 0, 0),
 datetime.datetime(2015, 10, 28, 0, 0),
 datetime.datetime(2015, 11, 13, 0, 0),
 datetime.datetime(2015, 11, 29, 0, 0),
 datetime.datetime(2015, 12, 15, 0, 0),
 datetime.datetime(2015, 12, 31, 0, 0),
 datetime.datetime(2016, 1, 16, 0, 0),
 datetime.datetime(2016, 2, 1, 0, 0),
 datetime.datetime(2016, 2